In [7]:
!pip install -r ../requirements.txt

  Using cached tokenizer-3.4.2-py2.py3-none-any.whl (79 kB)


### Read Data

In [8]:
import nltk
import re
import csv

# download the necessary nltk packages (only needs to be done once)
nltk.download('punkt')
nltk.download('stopwords')


def read_reviews_from_csv(file_path):
    ratings = []
    reviews = []
    with open(file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            # Assuming rating is the first item in each row, and review is the second item
            rating, review = row[0], row[1]
            ratings.append(int(rating)) # Convert rating to integer if needed
            reviews.append(review)
    return ratings, reviews


# function to clean a review text
def clean_review(review):
    # convert to lowercase
    review = review.lower()
    # remove non-alphanumeric characters
    review = re.sub(r'[^a-zA-Z0-9\s]', '', review)
    # remove extra whitespaces
    review = re.sub(r'\s+', ' ', review).strip()
    # return cleaned review
    return review


def get_ratings_sentences():
	ratings, reviews = read_reviews_from_csv("../data/reviews.csv")
	# clean the reviews and split them into sentences
	sentences = []
	new_ratings = []
	for idx, review in enumerate(reviews):
		# clean the review text
		cleaned_review = clean_review(review)
		# split the cleaned review into sentences
		review_sentences = nltk.sent_tokenize(cleaned_review)
		# append the sentences to the list
		sentences.extend(review_sentences)
		new_ratings.extend(ratings[idx])

	return sentences, new_ratings

### Call Model 
Instruct ATE + SA + Clustering

In [ ]:
import sys

# Add the parent directory of ABSA to the module search path
sys.path.append('..')

from models.SA import evaluate
from models.Clustering import scores
from scripts import script
sentences, new_ratings = get_ratings_sentences()

for sent in sentences:
	terms = script.return_iabsa(script.Task.ATE, "./Models/ate/allenaitk-instruct-base-def-pos-ate_check", 2, "The cab ride was amazing but the service was pricey")
	labels = evaluate(sent, terms)
	scores.get_clusters(terms, labels)